### Testing similarity between individuals

In [73]:
import random
import ast

In [46]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects) - 2)
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [148]:
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)), },# Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)), },# Delay
    2: { "drive_db": ('float', (1.0, 20.0)), },# Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },# Gain
    4: { "depth": ('float', (0.2, 0.6)), },# Phaser
    5: { "wet_level": ('float', (0.2, 0.6)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [10]:
individ1 = create_individual(effects, effect_structure)
individ2 = create_individual(effects, effect_structure)

print(individ1)
print(individ2)

# Extract common keys
common_keys = individ1.keys() & individ2.keys()

similarity_value = 0.0
for key in common_keys:
    weight = min(1 / len(individ1), 1 / len(individ2))
    #print(f'Key: {key}, weight: {weight}')
    
    inner_dict = effect_structure[key]
    lower_v = None
    max_v = None
    for param, value in inner_dict.items():
        lower_v = value[1][0]
        max_v = value[1][1]
                
    param_name = list(effect_structure[key].keys())[0]
    r = individ1[key][param_name]
    x = individ2[key][param_name]
    
    #print("r = ", r)
    #print("x = ", x)
    #print("low limit = ", lower_v)
    #print("max limit = ", max_v)
    
    distance_to_lower_v = abs(r - lower_v)
    distance_to_max_v = abs(r - max_v)
    
    limit_chosen = None
    if distance_to_lower_v < distance_to_max_v:
        limit_chosen = max_v
    else:
        limit_chosen = lower_v
    
    #print("abs(x - r) = ", abs(x - r))
    #print("abs(r - limit_chosen) = ", abs(r - limit_chosen))
    
    obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
    similarity_value += weight * obtained_value_normalized
    
print(similarity_value)

{0: {'rate_hz': 75.78}}
{5: {'wet_level': 0.32}, 0: {'rate_hz': 43.73}, 2: {'drive_db': 7.99}}
0.19235506290138119


In [147]:
def similarity_individuals(individ1, individ2):
    common_keys = individ1.keys() & individ2.keys()
    similarity = 0.0
    for key in common_keys:
        weight = min(1 / len(individ1), 1 / len(individ2))        
        inner_dict = effect_structure[key]
        param_name, (range_name, (lower_v, max_v)) = next(iter(inner_dict.items()))
         
        param_name = list(effect_structure[key].keys())[0]
        r = individ1[key][param_name]
        x = individ2[key][param_name]
        
        limit_chosen = max_v if abs(r - lower_v) < abs(r - max_v) else lower_v
        
        obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
        similarity += weight * obtained_value_normalized
        
    return similarity

def similarity_individuals_effect_only(individ1, individ2):
    common_keys = individ1.keys() & individ2.keys()
    similarity = 0.0
    for _ in common_keys:
        weight = min(1 / len(individ1), 1 / len(individ2)) 
        similarity += weight
    
    return similarity

In [7]:
individ1 = {3: {'gain_db': -1.5}, 1: {'delay_seconds': 1.62}}
individ2 = {1: {'delay_seconds': 1.62}}
similarity = similarity_individuals(individ1, individ2)

print(individ1)
print(individ2)
print(similarity)

{3: {'gain_db': -1.5}, 1: {'delay_seconds': 1.62}}
{1: {'delay_seconds': 1.62}}
0.5


## Test new similarity

In [151]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

a = np.array([1, 1, 1, 1, 1, 1])
b = np.array([0, 0, 0, 0, 0, 0])

s_max = np.sqrt(np.sum(np.square(a - b))) #similarità euclidea

a = np.array([0.5, 0, 0, 0, 0, 0])
b = np.array([0.2, 0.1, 0.1, 0.1, 0.1, 0.1])

s = np.sqrt(np.sum(np.square(a - b)))


print((s_max - s)/ s_max)



0.8472474768348054


In [49]:
import numpy as np

a = {0: {'rate_hz': 4.84}, 2: {'drive_db': 1.12}, 4: {'depth': 0.48}, 5: {'wet_level': 0.2  }}

effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)) },  # Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)) },  # Delay
    2: { "drive_db": ('float', (1.0, 20.0)) },  # Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },  # Gain
    4: { "depth": ('float', (0.2, 0.6)) },  # Phaser
    5: { "wet_level": ('float', (0.2, 0.6)) },  # Reverb
}

# Function to normalize a value within a given range
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# Initialize an array with zeros for all effects
B = np.zeros(n_effects)

# Iterate over each effect in 'a'
for effect_id, params in a.items():
    for param_name, param_value in params.items():
        _, (min_val, max_val) = effect_structure[effect_id][param_name]
        # Normalize the parameter value
        normalized_value = normalize(param_value, min_val, max_val)
        # Assign the normalized value to the corresponding position in B
        B[effect_id] = normalized_value

# Convert B to a numpy array A
A = np.array(B)

print("Normalized array A:", A)


Normalized array A: [0.20210526 0.         0.00631579 0.         0.7        0.        ]


In [140]:
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def similarity_array(individ, effect_structure, n_effects):
    sim_array = np.zeros(n_effects)

    for effect_id, params in individ.items():
        for param_name, param_value in params.items():
            
            if effect_id == 3 and param_name == 'gain_db':
                param_value += 10  # Apply the translation
                
            _, (min_val, max_val) = effect_structure[effect_id][param_name]
            # Normalize the parameter value
            normalized_value = normalize(param_value, min_val, max_val)
            # Assign the normalized value to the corresponding position in B
            sim_array[effect_id] = normalized_value

    # Convert B to a numpy array A
    return np.array(sim_array)

In [129]:
a = {0: {'rate_hz': 4.84}, 2: {'drive_db': 1.12}, 4: {'depth': 0.48}, 5: {'wet_level': 0.2  }}

effect_structure = {
    0: { "rate_hz": ('float', (1.0, 20.0)) },  # Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)) },  # Delay
    2: { "drive_db": ('float', (1.0, 20.0)) },  # Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },  # Gain
    4: { "depth": ('float', (0.2, 0.6)) },  # Phaser
    5: { "wet_level": ('float', (0.2, 0.6)) },  # Reverb
}


print(similarity_array(a, effect_structure, len(effect_structure)))

[0.20210526 0.         0.00631579 0.         0.7        0.        ]


In [141]:
def similarity_calculation(sim_array1, sim_array2, sim_max):
    sim = np.sqrt(np.sum(np.square(sim_array1 - sim_array2)))
    return (sim_max - sim)/ sim_max

In [145]:
ones = np.ones(6)  
zeros = np.zeros(6)

sim_max = np.sqrt(np.sum(np.square(ones - zeros))) #similarità euclidea

effect_structure = {
    0: { "rate_hz": ('float', (0.0, 20.0)) },  # Chorus
    1: { "delay_seconds": ('float', (0.0, 5.0)) },  # Delay
    2: { "drive_db": ('float', (0.0, 20.0)) },  # Distortion
    3: { "gain_db": ('float', (0, 20.0)) },  # Gain
    4: { "depth": ('float', (0.0, 0.6)) },  # Phaser
    5: { "wet_level": ('float', (0.0, 0.6)) },  # Reverb
}

'''
    0: { "rate_hz": ('float', (1.0, 20.0)) },  # Chorus
    1: { "delay_seconds": ('float', (1.0, 5.0)) },  # Delay
    2: { "drive_db": ('float', (1.0, 20.0)) },  # Distortion
    3: { "gain_db": ('float', (-10.0, 10.0)) },  # Gain
    4: { "depth": ('float', (0.2, 0.6)) },  # Phaser
    5: { "wet_level": ('float', (0.2, 0.6)) },  # Reverb
'''

individ1 = {3: {'gain_db': 8.18}}
individ2 = {3: {'gain_db': -6}}

sim_array1 = similarity_array(individ1, effect_structure, len(effect_structure))
sim_array2 = similarity_array(individ2, effect_structure, len(effect_structure))
print(sim_array1)
print(sim_array2)

print(similarity_calculation(sim_array1, sim_array2, sim_max))

[0.    0.    0.    0.909 0.    0.   ]
[0.  0.  0.  0.2 0.  0. ]
0.710551962061121


### Testing similarity on Dataset

In [152]:
import pandas as pd

df = pd.read_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search.csv')
similarities = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['GA_effects']

    # Convert the string representations to dictionaries
    if pd.notnull(applied_effects) and pd.notnull(ga_effects):
        try:
            applied_effects = ast.literal_eval(applied_effects)
            ga_effects = ast.literal_eval(ga_effects)
        except (ValueError, SyntaxError) as e:
            print(f"Error converting row {index}: {e}")
            similarities.append(None)
            continue

        # Perform the similarity calculation
        similarities.append(similarity_individuals(applied_effects, ga_effects))
    else:
        # If either field is missing, append a None or other placeholder value
        similarities.append(None)
        
# Add the 'similarity_indv' column to the DataFrame
df['similarity_indv'] = pd.Series(similarities)

# Save the updated DataFrame back to a CSV file
df.to_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search.csv', index=False)

In [23]:
df = pd.read_csv('./results/dataset_audios_guitar_lowR_1effect.csv')

df.insert(df.columns.get_loc('GA_effects_OR') + 1, 'random_effects', '')
df.to_csv('./results/dataset_audios_guitar_lowR_1effect.csv', index=False)

### Test individual similarity

In [8]:
import pandas as pd

df = pd.read_csv('./results/dataset_audios_guitar_low_ranges_w_clean.csv')
similarities = []
similarities_effects_only = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['GA_effects']

    # Convert the string representations to dictionaries
    if pd.notnull(applied_effects) and pd.notnull(ga_effects):
        try:
            applied_effects = ast.literal_eval(applied_effects)
            ga_effects = ast.literal_eval(ga_effects)
        except (ValueError, SyntaxError) as e:
            print(f"Error converting row {index}: {e}")
            similarities.append(None)
            continue

        # Perform the similarity calculation
        similarities.append(similarity_individuals(applied_effects, ga_effects))
        similarities_effects_only.append(similarity_individuals_effect_only(applied_effects, ga_effects))
    else:
        # If either field is missing, append a None or other placeholder value
        similarities.append(None)
        similarities_effects_only.append(None)
        
# Add the 'similarity_indv' column to the DataFrame
df['similarity_indv'] = pd.Series(similarities)
df['similarity_indv_effects_only'] = pd.Series(similarities_effects_only)

# Save the updated DataFrame back to a CSV file
df.to_csv('./results/dataset_audios_guitar_low_ranges_w_clean.csv', index=False)

In [126]:
import pandas as pd

df = pd.read_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search.csv')
df['GA_effects'] = pd.Series('')
df['similarity_indv'] = pd.Series('')
df.to_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search.csv', index=False)

### Testing NEW similarity on Dataset

In [153]:
import pandas as pd

df = pd.read_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search_new_diss.csv')

effect_structure = {
    0: { "rate_hz": ('float', (0.0, 20.0)) },  # Chorus
    1: { "delay_seconds": ('float', (0.0, 5.0)) },  # Delay
    2: { "drive_db": ('float', (0.0, 20.0)) },  # Distortion
    3: { "gain_db": ('float', (0, 20.0)) },  # Gain
    4: { "depth": ('float', (0.0, 0.6)) },  # Phaser
    5: { "wet_level": ('float', (0.0, 0.6)) },  # Reverb
}

ones = np.ones(6)  
zeros = np.zeros(6)

sim_max = np.sqrt(np.sum(np.square(ones - zeros))) #similarità euclidea

similarities = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['GA_effects']

    # Convert the string representations to dictionaries
    if pd.notnull(applied_effects) and pd.notnull(ga_effects):
        try:
            applied_effects = ast.literal_eval(applied_effects)
            ga_effects = ast.literal_eval(ga_effects)
        except (ValueError, SyntaxError) as e:
            print(f"Error converting row {index}: {e}")
            similarities.append(None)
            continue
        
        sim_array1 = similarity_array(applied_effects, effect_structure, len(effect_structure))
        sim_array2 = similarity_array(ga_effects, effect_structure, len(effect_structure))

        similarities.append(similarity_calculation(sim_array1, sim_array2, sim_max))
        
# Add the 'similarity_indv' column to the DataFrame
df['similarity_indv'] = pd.Series(similarities)

# Save the updated DataFrame back to a CSV file
df.to_csv('./results/dataset_audios_guitar_low_ranges_w_clean_random_search_new_diss.csv', index=False)